<a href="https://colab.research.google.com/github/jiin124/review_project/blob/main/%EC%95%84%EA%B3%A0%EB%8B%A4_%ED%98%B8%ED%85%94%EB%A6%AC%EB%B7%B0_%EB%8B%A8%EC%96%B4%EB%B9%88%EB%8F%84%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [2]:
import io
import pandas as pd
import csv
from konlpy.tag import Komoran
from collections import defaultdict

In [3]:
df=pd.read_csv('/content/hotel_review_clean.csv')
df.head()

,star,Hotel_name,review_rate,review_grade,reviewer_name,trip_type,room_type,trip_date_year,trip_date_month,title,review,trip_day,review_date,datetime,rate
0,1,유원호텔_서초,9.2,최고,jeon,나홀로 여행객,스탠다드 더블 베드,2020.0,12.0,친절했습니다.”,아버님 서울 병원 방문 목적으로 혼자 숙박하셔야 되어 걱정 많았으나 사전 전화 문의...,1,2021-01-01,2021-01-01,4
1,1,유원호텔_서초,7.2,좋음,Joonvum,출장 여행객,스탠다드 더블 베드,2019.0,6.0,유원호텔 이용후기”,3호선 남부터미널 역이 매우 가깝고 교대역까지도 도보로 이동 가능한 거리에 위치함....,1,2019-07-07,2019-07-07,3
2,1,유원호텔_서초,8.8,우수,JaeYoung,나홀로 여행객,디럭스 더블베드,2019.0,9.0,대체적으로 만족합니다”,디럭스 더블베드 룸에 숙박했습니다. 방은 깨끗했고 직원분들도 친절했습니다. 남부터미...,1,2019-10-01,2019-10-01,4
3,1,유원호텔_서초,9.0,최고,Hye,그룹 여행객,스탠다드룸,2016.0,1.0,실내 조명(전등) 좀 환하게...”,숙박의 목적에 따라 평가가 달라지겠지만 저희는 5명 친구들과 함께 하는 거라 도란도...,1,2016-01-19,2016-01-19,4
4,1,유원호텔_서초,10.0,최고,SUNGMIN,커플/2인 여행객,스탠다드 더블 베드,2012.0,8.0,가격대비 위치 최고의 모텔”,일단 남부터미널에서 가깝고 놀라울 만큼 저렴한 숙박료에 감탄했습니다. 지방에 사는지...,1,2012-08-23,2012-08-23,5


In [10]:
reviews=df['review']
reviews.tail()

3012         업그레이드된 모텔
3013    친구와 서울 여행 중 들림
3014    개조된 모텔의 느낌이...
3015           잘 이용하네요
3016    조금은 밝은 분위기의 모텔
Name: review, dtype: object

In [5]:
komoran = Komoran(userdic='/content/user_dictionary.txt') # userdic 추가

#고려하는 품사로만 tag로 지정
'''
긍정지정사(VCP), 부정지정사(VCN)
일반명사(NNG), 고유명사(NNP), 동사(VV), 형용사(VA), 부사(MAG)
수사(NR), 감탄사(IC), 어근(XR), 보조 용언(VX)
'''

In [7]:
tag_list = ['VCP','VCN',
            'NNG','NNP','VV','VA','MAG',
            'NR','IC','XR','VX']

In [9]:
# 계산할 tag만 남기고 다른 품사는 제거
def remove_word_by_tag(token,tag_list):
  temp = list()
  for k in range(len(token)):
    if token[k][1] in tag_list:
      temp.append(token[k])
    else:
      continue
  return temp

  # 형태소 분석
def morph_analysis(comments):
  morph_token = []
  for comment in comments:
    token = komoran.pos(comment)
    token = remove_word_by_tag(token, tag_list)
    morph_token.append(token)
  return morph_token


# 품사별 단어 빈도수 계산
def word_count(part, tokens): # part는 원하는 품사
  part_list = defaultdict(int)
  for token in tokens:
    for k in range(len(token)):
      if token[k][1] == part: # token의 품사가 지정한 품사이면
        part_list[token[k][0]] += 1
  return part_list

In [11]:
i = 1
for review in reviews:
  token = komoran.pos(review)
  print(i,'번째:', token)
  i += 1

header_list = ['count','WORD','TAG','reinforcer','score']
df = pd.DataFrame(columns=header_list)

morph_token = morph_analysis(reviews)
word_list = []
for tag in tag_list:
  word_list = word_count(tag, morph_token)
  sorted_list = sorted(word_list.items(), key=lambda x: x[1], reverse=True)
  for i in range(len(sorted_list)):
    df = df.append({'count':sorted_list[i][1], 'WORD':sorted_list[i][0],'TAG':tag,'reinforcer':1, 'score':0}, ignore_index=True)
    df = df.sort_values('count', ascending=False)
print(df)

df.to_csv('score_dic.csv')

1 번째: [('아버님', 'NNG'), ('서울', 'NNP'), ('병원', 'NNG'), ('방문', 'NNG'), ('목적', 'NNG'), ('으로', 'JKB'), ('혼자', 'NNG'), ('숙박', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('어야', 'EC'), ('되', 'VV'), ('어', 'EC'), ('걱정', 'NNG'), ('많', 'VA'), ('았', 'EP'), ('으나', 'EC'), ('사전', 'NNG'), ('전화', 'NNG'), ('문의', 'NNG'), ('에', 'JKB'), ('도', 'JX'), ('친절', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('었', 'EP'), ('고', 'EC'), ('사용', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('ㄴ', 'ETM'), ('아버님', 'NNG'), ('도', 'JX'), ('매우', 'MAG'), ('만족', 'NNG'), ('하', 'XSV'), ('시', 'EP'), ('었', 'EP'), ('습니다', 'EF'), ('.', 'SF')]
2 번째: [('3호선', 'NNP'), ('남부터미널', 'NNP'), ('역', 'NNG'), ('이', 'JKS'), ('매우', 'MAG'), ('가깝', 'VA'), ('고', 'EC'), ('교대역', 'NNP'), ('까지', 'JX'), ('도', 'JX'), ('도보', 'NNG'), ('로', 'JKB'), ('이동', 'NNP'), ('가능', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('거리', 'NNG'), ('에', 'JKB'), ('위치', 'NNG'), ('하', 'XSV'), ('ㅁ', 'ETN'), ('.', 'SF'), ('싸', 'VA'), ('ㄴ', 'ETM'), ('가격', 'NNG'), ('.', 'SF'), ('같', 'VA'), ('은', 'ETM'), ('가격', 'NNG'), ('에',